In [1]:
import json
from urllib.request import urlopen, Request
from urllib.parse import urlencode

In [49]:
def lambda_handler(event=None, context=None):
    # requests pb2020 api
    print('fetching data')
    content = urlopen('https://raw.githubusercontent.com/2020PB/police-brutality/data_build/all-locations.json').read().decode('utf-8')
    alldata = json.loads(content)
    print("data:",type(alldata['data']),len(alldata['data']))
    data = alldata['data']
    for i in data:
      # check if incident id exists in db
      incident_id = i['id']
      print("connecting to ds api")
      incident_url = f'http://human-rights-considered.eba-api7kmju.us-east-1.elasticbeanstalk.com/incidents/{incident_id}'
      incident_response = urlopen(incident_url).read().decode()
      if incident_response.status_code != 400:
        print('creating incident')
        create_incident_url = f'http://human-rights-considered.eba-api7kmju.us-east-1.elasticbeanstalk.com/incidents/'
        incident_fields = dict(
            incident_id = incident_id,
            incident_description = i['name'],
            time_id = i['date']
        )
        incident_request = Request(create_incident_url, urlencode(incident_fields).encode())
        incident_json = urlopen(incident_request).read().decode()
        print("created:",incident_json)

        print('creating evidence')
        evidence_url = f'http://human-rights-considered.eba-api7kmju.us-east-1.elasticbeanstalk.com/incidents/{incident_id}/evidence/'
        evidence_fields = dict(
            link_string = i['links'][0]
        )
        evidence_request = Request(evidence_url, urlencode(evidence_fields).encode())
        evidence_json = urlopen(evidence_request).read().decode()
        print('created:',evidence_json)

        print('creating place')
        place_url = f'http://human-rights-considered.eba-api7kmju.us-east-1.elasticbeanstalk.com/incidents/{incident_id}/place/'
        place_fields = dict(
            city = i['city'],
            state = i['state']
        )
        place_request = Request(place_url, urlencode(place_fields).encode())
        place_json = urlopen(place_request).read().decode()
        print("created:",place_json)





In [50]:
lambda_handler()

fetching data
data: <class 'list'> 1070
connecting to ds api


HTTPError: ignored